In [8]:
import pandas as pd

In [11]:
def find_basepairs(s):
    stack = []
    result = []

    for index, char in enumerate(s):
        if char == '(':
            stack.append(index)
        elif char == ')':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                result.append((open_index, index))

    return sorted(result)

def find_knotpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '[':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ']':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_otherpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '{':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == '}':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_pairs(s):
    basepairs = find_basepairs(s)
    knotpairs = find_knotpairs(s)
    otherpairs = find_otherpairs(s)
    pairs = basepairs + knotpairs + otherpairs
    return sorted(pairs)



# def possible_pairs(s):

#     l = len(s)
#     pairs = []
#     for i in range(l):
#         for j in range(i+1,l):
#             pair = (i, j)
#             pairs.append(pair)

#     return pairs

#faster benchmark which do not calculate accuracy and TN
def Full_Benchmark_2Dv2(pred, ref, v=True):
    
    #Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1
    taminoto = tcounter/len(ref)


    #Binary classification metrics for all pairs
    pred_pairs = find_pairs(pred)
    ref_pairs = find_pairs(ref)
    # poss_pairs = possible_pairs(ref)
    
    # ref_notpairs = [x for x in poss_pairs if x not in ref_pairs] 
    # pred_notpairs = [x for x in poss_pairs if x not in pred_pairs]
    

    TP = len(set(pred_pairs).intersection(ref_pairs))
    FP = len(pred_pairs)-TP
    # TN = len(set(pred_notpairs).intersection(ref_notpairs)) 
    # FN = len(pred_notpairs)-TN
    FN = len(ref_pairs) -TP

    if TP == 0:
        TP = 0.0001

    if FP == 0:
        FP = 0.0001

    # if TN == 0:
    #     TN = 0.0001

    if FN == 0:
        FN = 0.0001
    
    # print(TP, TN, FP, FN, TP+FP+TN+FN)
    # print(len(pred_pairs), len(pred_notpairs), len(ref_pairs), len(ref_notpairs), len(poss_pairs))

    # Accuracy = (TP+TN)/(TP+TN+FP+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1_score = (2*Precision*Recall)/(Precision+Recall)

    
    if v:
        print('Taminoto accuracy = ', round(taminoto, 3))
        # print('Accuracy =', round(Accuracy, 3))
        print('Precision =', round(Precision, 3))
        print('Recall =', round(Recall, 3))
        print('F1-score =', round(F1_score, 3))

    else:
        return taminoto, Precision, Recall, F1_score

#Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
def taminoto(pred, ref):
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1  

    return tcounter/len(ref)


def taminoto_accurate_percent(preds, refs, cutoff=0.85, v=True):
    
    acc_list = []
    acc = 0
    acc_counter = 0
    for i in range(len(preds)):
        acc = taminoto(preds[i], refs[i])
        if acc >= cutoff:
            acc_counter += 1
        acc_list.append(acc)

    if v:
        print(f'% of Accurate structures (taminoto acc. >= {cutoff}) =', round(100*acc_counter/len(preds), 2))

    else:
        return acc_counter/len(preds), acc_list

In [16]:
!mxfold2 predict test.fa
# >DS4440
# GGAUGGAUGUCUGAGCGGUUGAAAGAGUCGGUCUUGAAAACCGAAGUAUUGAUAGGAAUACCGGGGGUUCGAAUCCCUCUCCAUCCG
# (((((((........(((((..((((.....))))...)))))...................(((((.......)))))))))))). (24.8)

>DS4440
GGATGGATGTCTGAGCGGTTGAAAGAGTCGGTCTTGAAAACCGAAGTATTGATAGGAATACCGGGGGTTCGAATCCCTCTCCATCCG
............................(((.........)))...................(.(((.......))).)........ (9.3)


In [3]:
!mxfold2 predict data.fasta > mxfold2.fasta

In [4]:
def fastaparser(filename):

    with open(filename, 'r') as fasta:
        lines = fasta.readlines()
        preds = []
        for i in range(2, len(lines), 3):
            pred = lines[i].split()
            preds.append(pred[0])

    return preds

mxfold_preds = fastaparser('mxfold2.fasta')
len(mxfold_preds)

354

In [9]:
db = pd.read_csv('data.csv')
db['mxfold2'] = pd.Series(mxfold_preds)

In [12]:
#RNA+DNA
pred = ''.join(list(db['mxfold2']))
ref2d = ''.join(list(db['2D']))
Full_Benchmark_2Dv2(pred, ref2d)

Taminoto accuracy =  0.806
Precision = 0.81
Recall = 0.745
F1-score = 0.776


In [13]:
rnadb = db[db['NAtype'] == 'RNA']
dnadb = db[db['NAtype'] == 'DNA']

In [14]:
print('RNA')
pred = ''.join(list(rnadb['mxfold2']))
ref2d = ''.join(list(rnadb['2D']))
Full_Benchmark_2Dv2(pred, ref2d)

RNA
Taminoto accuracy =  0.818
Precision = 0.817
Recall = 0.763
F1-score = 0.789


In [15]:
print('DNA')
pred = ''.join(list(dnadb['mxfold2']))
ref2d = ''.join(list(dnadb['2D']))
Full_Benchmark_2Dv2(pred, ref2d)

DNA
Taminoto accuracy =  0.674
Precision = 0.677
Recall = 0.484
F1-score = 0.564
